In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
variable_names = ['10yr', '30yr', '60yr']

In [ ]:
# out put the forced trend and pvalue
dir_in ='./Figure3/ACCESS/'

# trend_annual_da = {}
# p_val_annual_da = {}

# for var in variable_names:
#     trend_annual_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_unforced_' + var + '_trend.nc', chunks={'run': 1})


In [ ]:
# for var in variable_names:
#     trend_annual_da[var] = trend_annual_da[var].rename({'__xarray_dataarray_variable__': 'trend'})

In [ ]:
# input the unforced trend and pvalue
unforced_trend_std_da = {}
# unforced_p_val_da = {}

for var in variable_names:
    unforced_trend_std_da[var] = xr.open_mfdataset(dir_in + 'ICV_segments_' + var + '_std_trend_pattern.nc', chunks={'run': 1})
    # unforced_p_val_da[var] = xr.open_mfdataset(dir_in + 'ACCESS_unforced_' + var + '_pvalue.nc', chunks={'run': 1})

In [ ]:
unforced_trend_std_da

In [ ]:
unforced_trend_std_da['10yr']

### calculate the pattern correlation

In [ ]:
# Input the 10-year, 30-year and 60-year ensemble mean SAT trend data
dir_input = './Supp_Figure7/data/'

ACCESS_10yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_10yr_noise_trend_std.nc')
ACCESS_30yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_30yr_noise_trend_std.nc')
ACCESS_60yr_trend   = xr.open_dataset(dir_input + 'ACCESS_annual_60yr_noise_trend_std.nc')

In [ ]:
ACCESS_10yr_trend

In [ ]:
unforced_trend_std_da['10yr']['ICV_segments_10yr_std_trend_pattern'].isel(run=0).values

In [ ]:
# pattern correlation betwenn observed forced pattern vs. Model simulated forced pattern
import scipy.stats as stats
# 10yr unforced trend in each realization .vs. ACCESS ensemble stddev. trend ACCESS_10yr_trend

ACCESS_unforced_10yr_trend = unforced_trend_std_da['10yr']['ICV_segments_10yr_std_trend_pattern']
ACCESS_unforced_30yr_trend = unforced_trend_std_da['30yr']['ICV_segments_30yr_std_trend_pattern']
ACCESS_unforced_60yr_trend = unforced_trend_std_da['60yr']['ICV_segments_60yr_std_trend_pattern']

# Pattern correlations calculation
trend_pattern_correlation_10yr = []
for i in range(len(unforced_trend_std_da['10yr'].run)):
    trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_unforced_10yr_trend.isel(run=i).values.flatten(), ACCESS_10yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_30yr = []
for i in range(len(unforced_trend_std_da['30yr'].run)):
    trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_unforced_30yr_trend.isel(run=i).values.flatten(), ACCESS_30yr_trend['tas'].values.flatten())[0])

trend_pattern_correlation_60yr = []
for i in range(len(unforced_trend_std_da['60yr'].run)):
    trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_unforced_60yr_trend.isel(run=i).values.flatten(), ACCESS_60yr_trend['tas'].values.flatten())[0])
    

In [ ]:
trend_pattern_correlation_60yr

In [ ]:
# save the pattern correlation into text file
# Save to a text file
with open('pattern_correlations_unforced_STD.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# Input the observational forced and unforced trend wrt. ACCESS-CM6A-LR
dir_obs = './Figure2/data/'

Obs_forced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_10yr_trend.nc')
Obs_forced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_30yr_trend.nc')
Obs_forced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_forced_60yr_trend.nc')

Obs_unforced_10yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_10yr_trend.nc')
Obs_unforced_30yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_30yr_trend.nc')
Obs_unforced_60yr_trend = xr.open_dataset(dir_obs + 'HadCRUT5_annual_internal_60yr_trend.nc')

In [ ]:
Obs_unforced_10yr_trend

In [ ]:
Obs_forced_10yr_trend = Obs_forced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_30yr_trend = Obs_forced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_forced_60yr_trend = Obs_forced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})

Obs_unforced_10yr_trend = Obs_unforced_10yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_30yr_trend = Obs_unforced_30yr_trend.rename({'__xarray_dataarray_variable__':'tas'})
Obs_unforced_60yr_trend = Obs_unforced_60yr_trend.rename({'__xarray_dataarray_variable__':'tas'})


In [ ]:
# Save to a text file
with open('obs_MMEM_forced_vs_ACCESS_forced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
# Save to a text file
with open('obs_MMEM_unforced_vs_ACCESS_unforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")